# Characterize the sizes of low-z galaxies in the catalog

Characterize how many galaxies have sizes and what bands were used etc.

Figure out if any clean up of the catalog is needed.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

## Read the new list, with diameters

In [ ]:
fnm = "lowz_galaxies_with_diameters_fix.csv"
gal_name = np.loadtxt(fnm, dtype='str', usecols=(0), delimiter=',')
gal_ra = np.loadtxt(fnm, dtype='float', usecols=(1), delimiter=',')
gal_dec = np.loadtxt(fnm, dtype='float', usecols=(2), delimiter=',')
gal_z = np.loadtxt(fnm, dtype='float', usecols=(3), delimiter=',')

gal_ma = np.loadtxt(fnm, dtype='float', usecols=(4), delimiter=',')
gal_ar = np.loadtxt(fnm, dtype='float', usecols=(5), delimiter=',')
gal_pa = np.loadtxt(fnm, dtype='float', usecols=(6), delimiter=',')
gal_fr = np.loadtxt(fnm, dtype='float', usecols=(7), delimiter=',')

In [ ]:
print(len(gal_name))

## Galaxies without size and shape

In [ ]:
tx = np.where(gal_ma < 0.0)[0]
print(len(tx), '(' + str(np.round(100.0 * len(tx)/len(gal_name), 1)) + '%) have no diameter.')

In [ ]:
tx = np.where(gal_ar < 0.0)[0]
print(len(tx), '(' + str(np.round(100.0 * len(tx)/len(gal_name), 1)) + '%) have no axis ratio.')

In [ ]:
tx = np.where(gal_pa < 0.0)[0]
print(len(tx), '(' + str(np.round(100.0 * len(tx)/len(gal_name), 1)) + '%) have no position angle.')

In [ ]:
fig = plt.figure(figsize = (6, 3))
plt.plot(gal_ra, gal_dec, 'o', ms=2, mew=0, color='lightgrey', label='all')
tx = np.where(gal_ma < 0.0)[0]
plt.plot(gal_ra[tx], gal_dec[tx], 'o', ms=2, mew=0, color='darkred', label='no size')
plt.xlabel('RA')
plt.ylabel('Dec')
plt.legend(loc='best')
plt.show()

In [ ]:
fig = plt.figure(figsize=(3, 2))
plt.hist(gal_z, log=True, color='lightgrey', label='all')
tx = np.where(gal_ma < 0.0)[0]
plt.hist(gal_z[tx], log=True, color='darkred', label='no size')
plt.xlabel('redshift')
plt.legend(loc='best')
plt.show()

## Distribution of size parameters

### Major axis

In [ ]:
fig = plt.figure(figsize=(3, 2))
plt.hist(gal_ma, log=True)
plt.xlabel('major axis')
plt.show()

What's up with the huge one?

In [ ]:
tx = np.where(gal_ma > 5000)[0]
for x in tx: 
    print(gal_name[x], gal_ma[x])

Show the LMC is not in the catalog.

In [ ]:
for name in gal_name:
    if name.find('Magellanic') >= 0:
        print(name)

The distribution without the huge ones.

In [ ]:
tx = np.where(gal_ma < 5000)[0]
fig = plt.figure(figsize=(6, 2))
plt.hist(gal_ma[tx], bins=60, log=True)
plt.xlabel('major axis')
plt.show()

### Position angle

In [ ]:
tx = np.where((gal_ma < 5000) & (gal_ma > 0.0))[0]
fig = plt.figure(figsize=(6, 2))
plt.hist(gal_pa[tx], bins=60, log=True)
plt.xlabel('position angle')
plt.show()

In [ ]:
tx = np.where((gal_ma < 5000) & (gal_ma > 0.0) & (gal_pa > 300))[0]
for x in tx:
    print('%20s %6.1f %6.1f' % (gal_name[x], gal_ma[x], gal_pa[x]))

### Axis ratio

In [ ]:
tx = np.where((gal_ma < 5000) & (gal_ma > 0.0))[0]
fig = plt.figure(figsize=(6, 2))
plt.hist(gal_ar[tx], bins=60, log=True)
plt.xlabel('axis ratio')
plt.show()

## Duplicate and overlapping galaxies

There are groups and interacting galaxies, and also duplicates.

For example, NGC 1741 is a group (NCG 1741A, B) and there are duplicates in NED (NED02, NED03)

In [ ]:
os.system("grep 'NGC 1741' lowz_galaxies_with_diameters_fix.csv")

For each galaxy, sum the number of neighbors within a radius set by its semi-major axis.

This measure of "overlap" is an overestimate since we're using semi-major axis as radius, and not considering ellipticity, orientation, etc.

In [ ]:
gal_nover = np.zeros(len(gal_name), dtype='int')
for i, name in enumerate(gal_name):
    temp = np.sqrt(((np.cos(np.deg2rad(gal_dec[i]))) * (gal_ra[i] - gal_ra))**2 + 
                   (gal_dec[i] - gal_dec)**2)
    tx = np.where((temp > 0) & (temp < gal_ma[i]/3600.0/2.0))[0]
    gal_nover[i] = len(tx)
    del temp, tx  

In [ ]:
tx = np.where(gal_nover > 0.0)[0]
print(len(tx), '(' + str(np.round(100.0 * len(tx)/len(gal_name), 1)) + \
      '%) galaxies have >0 overlapping galaxies.')

In [ ]:
fig = plt.figure(figsize=(6, 2))
plt.hist(gal_nover, bins=np.max(gal_nover), log=True)
plt.xlabel('N overlap')
plt.show()

What's up with the one galaxy with 15 overlaps?

In [ ]:
tx = np.where(gal_nover > 10)[0]
for x in tx:
    print(gal_name[x], gal_ma[x])

In [ ]:
os.system("grep 'NGC 1399' lowz_galaxies_with_diameters_fix.csv")

In [ ]:
fig = plt.figure(figsize=(3, 2))
plt.plot(gal_ma, gal_nover, 'o', alpha=0.3)
plt.xlabel('major axis')
plt.ylabel('N overlap')
plt.show()

fig = plt.figure(figsize=(3, 2))
plt.plot(gal_ma, gal_nover, 'o', alpha=0.3)
plt.xlim([-10, 510])
plt.ylim([-1, 6])
plt.xlabel('major axis')
plt.ylabel('N overlap')
plt.show()

The number of overlaps isn't correlated with major axis so it's not just that the bigger ones have more 'chance' alignments. The overlaps are thus probably mostly duplicates and groups/pairs of galaxies.

In [ ]:
tx = np.where((gal_nover > 0) & (gal_ma < 0.0))[0]
print(len(tx))

Above shows none of the galaxies with overlaps were missing a major axis.

This means that there is no chance of an alert being cross-matched to a galaxy that is a duplicate without a size, where it could have been matched to a duplicate WITH a size.

## Last step

Copy the fixed file to final/ and create a readme.